# Lab 5. Parte "a": El genoma te da sorpresas, sorpresas te da el genoma


La __replicación__ del genoma es una de las tareas más importantes que efectúa una célula para poder dividirse.
Dicha replicación comienza en una región del genoma conocida como el origen de replicación (ori) y es realizada por las ADN polimerasas. 

Localizar el origen de replicación es una tarea importante para comprender no sólo cómo las células se replican sino también para la creación de fármacos, vacunas, etc.


**Pregunta: ¿Cómo tratarías de encontrar (experimentalmente) el origen de replicación en un genoma de una bacteria?**


Por simplicidad, vamos a centrarnos en la tarea de encontrar ori en genomas de bacterias, los cuales principalmente consisten de un sólo cromosoma circular. Para ello, vamos a estudiar una bacteria de la que se conoce su ori, para después determinar qué es lo que tiene de especial esta región del genoma. 

Ya hemos visto que el origen de replicación del Vibrio Cholerae es:

In [1]:
fd = open("./oric.fasta")
vc_oric = fd.readlines()[1]
fd.close()
print(len(vc_oric))
print(vc_oric)

541
atcaatgatcaacgtaagcttctaagcatgatcaaggtgctcacacagtttatccacaacctgagtggatgacatcaagataggtcgttgtatctccttcctctcgtactctcatgaccacggaaagatgatcaagagaggatgatttcttggccatatcgcaatgaatacttgtgacttgtgcttccaattgacatcttcagcgccatattgcgctggccaaggtgacggagcgggattacgaaagcatgatcatggctgttgttctgtttatcttgttttgactgagacttgttaggatagacggtttttcatcactgactagccaaagccttactctgcctgacatcgaccgtaaattgataatgaatttacatgcttccgcgacgatttacctcttgatcatcgatccgattgaagatcttcaattgttaattctcttgcctcgactcatagccatgatgagctcttgatcatgtttccttaaccctctattttttacggaagaatgatcaagctgctgctcttgatcatcgtttc



¿Cómo sabe la célula bacteriana que tiene que comenzar la replicación en esta pequeña región (\~500bp o 0.0487%) del genoma (\~1.000.000bp)? ¡Ha de existir algún tipo de mensaje que le indique dónde mirar! De hecho, el comienzo de la replicación es soportado por una proteína llamada DnaA que va a adherirse a un pequeño segmento en ori (caja DnaA). 

La caja DnaA es un mnensaje en la secuencia que le dice a la proteína DnaA: "¡Eh, empieza por aquí!" 

La pregunta es: ¿Cómo podemos encontrar ese mensaje sin saber apenas nada sobre él? ¿Qué es lo que lo hace especial? 

Para dar con la respuesta, vamos a emplear métodos estadísticos y cuantitativos que nos van a permitir caracterizar de manera clara estas particularidades.

## ¿Cómo de sorprendente?
Debido a diferentes procesos biológicos, se sabe que ciertas cadenas de nucleótidos son _sorprendentemente_ frecuentes en algunas regiones del genoma. La causa es que comúnmente ciertas proteínas sólo pueden unirse al ADN si existen ciertas secuencias de nucleótidos. Cuantas más de estas cadenas existan, más probabilidades hay de que esta unión ocurra. 

Pero, ¿qué significa _"sorprendentemente"_? ¡Afortunadamente, la estadística existe precisamente para cuantificar la sorpresa (entre otras cosas)!

Por ejemplo, considera la secuencia

`"ACAACTATGCATACTATCGGGAACTATCCT"`

¿Dirías que existe una subcadena ("k-mero" o secuencia de k nucleótidos) que sea _sorprendentemente_ frecuente? ¿Por qué?

__Nota__: Un k-mero, del griego méros (parte), se podría traducir como "una parte o porción de longitud k". 
"-mer" es un sufijo muy utilizado en química para denotar partes de una molécula (por ej, "polímero"). 

In [2]:
# Vamos a escribir un método
seq_1 = "ACAACTATGCATACTATCGGGAACTATCCT"

def cuenta_patron(secuencia, patron):
    cuenta = 0 #Aquí guardo las veces que encuentro el patrón. 
    # Voy moviéndome una posición a la derecha y comprobando si los siguientes caracteres siguientes son iguales.
    for i in range(len(secuencia) - len(patron) + 1): 
        if secuencia[i:i+len(patron)] == patron: 
            cuenta += 1
    return cuenta

In [3]:
print(len(seq_1))
cuenta_patron(seq_1, "ACTAT")

30


3

Parece que el 5-mero ACTAT se repite 3 veces en la secuencia dada, que tiene longitud 30.

¿Pero, cómo de sorprendente es este hecho? Vamos a verlo con un poco de combinatoria. 

## Combinaciones y permutaciones

- __Permutación__: mezcla ordenada -> 473, 374, 347 __no__ son lo mismo
- __Combinación__: mezcla sin importar el orden -> 473, 374, 347 son lo mismo


Como ya hemos visto en el anterior lab, en ambos casos, podemos permitir que se repitan elementos o no. 

Vamos a verlo con ejemplos concretos para cada caso:

### Permutaciones con repetición de longitud k
Fáciles de calcular, simplemente tenemos que hacer k elecciones de n elementos, siendo n el número de opciones que tenemos para elegir. En el caso del alfabeto genómico, {A,G,T,C}, $n=4$. Por ejemplo, para generar todos los 5-meros posibles usando este alfabeto tendremos que considerar todas las posibilidades de hacer 5 elecciones aleatorias seguidas de uno de sus símbolos (4 en cada caso).

$n \times n \times n \times n \times n$ (5 veces) $= 4 \times 4 \times 4 \times 4 \times 4 = 4^{5} = 1024$.

De forma general, haremos diremos que el número de permutaciones con repetición de longitud k es $n^{k}$

Por tanto, como el k-mero en el que estamos pensando es `ACTAT`, la probabilidad de escogerlo entre los 1024 disponibles es: $\frac{1}{1024} \approx 0.00098$

### Permutaciones sin repetición de longitud k <= n

Este caso es igual al anterior, pero ahora nuestras opciones disponibles se reducen en 1 con cada eleccción. 

$n \times (n-1) \times (n-2) \times (n-3) \times (n-4) = 4 \times 3 \times 2 \times 1 = 5! = 24$

lo que nos da todas las permutaciones sin repetición de $n=4$ elementos. 

Por tanto, de manera general, el número de permutaciones posibles con n elementos será $n!$. 

__nota:__ _El factorial de un número entero positivo n se escribe n! y es el producto de todos los números enteros positivos desde 1 hasta n._

Imagina ahora que sólo nos interesan las permutaciones sin repetición de hasta 2 elementos. ¿Cómo lo haríamos?

Tendríamos que hacer $n - k = 4 - 2 = 2$ elecciones de este tipo.

$n \times (n-1) = 12$

entonces, como norma siempre llegaremos hasta el elemento $(n-k+1)$ en el factorial.

Por tanto, todo esto puede ser convenientemente rescrito para dar con la formula:

$\frac{n!}{(n-k)!}$

__Atent@! Sólo podemos escoger hasta $k \leq n$, ya que sí $k \gt n$, entonces estaríamos obligados a repetir elementos!__

Demostración empírica:

In [39]:
from itertools import permutations, product 
def print_results(it):
    for i in it:
        print(i)

In [53]:
permutaciones = list(permutations('ACGT', 2)) #genera todas las permutaciones de 4 elementos cogidos de 2 en 2.
print("Hay %d permutaciones" % len(permutaciones))
print_results(permutaciones)

Hay 12 permutaciones
('A', 'C')
('A', 'G')
('A', 'T')
('C', 'A')
('C', 'G')
('C', 'T')
('G', 'A')
('G', 'C')
('G', 'T')
('T', 'A')
('T', 'C')
('T', 'G')


### Combinaciones sin repetición de longitud k <= n
Para calcular las combinaciones (en las que no importa el orden), vamos a tomar todas las posibles permutaciones sin repetición para $k=n$, de las cuales tendremos que descartar las que están en diferente orden.

Fíjate en el código del anterior ejemplo, ¿cuántas tendríamos que descartar?

In [54]:
# Como ejemplo, vamos a calcular empíricamente cuántas de las permutaciones 
# que hemos obtenido en el paso anterior aparecen otra vez con sus elementos en distinto orden.
frecuencias = {}
for i in range(len(permutaciones)):
    conjunto_i = set(permutaciones[i])
    frecuencias[i] = 0
    for j in range(len(permutaciones)):
        conjunto_j = set(permutaciones[j])
        if conjunto_i == conjunto_j:
            frecuencias[i] += 1

In [55]:
frecuencias

{0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2, 10: 2, 11: 2}

Vaya! Cada una de las permutaciones aparece dos veces en distinto orden! 

__Pregunta:__ _¿Cuántas veces aparecería si estuviésemos manejando permutaciones de 3 elementos?_

Por tanto, como una permutación de k elementos puede ordenarse de $k!$ maneras diferentes, tenemos que divir el número de permutaciones por $k!$

De manera general:

$\frac{n!}{(n-k)!}\times\frac{1}{k!} = \frac{n!}{k!(n-k)!} = {n\choose k}$

### Combinaciones con repetición de longitud k
Explicar cómo se obtiene el número de combinaciones con repetición es un poco más complicado y no lo vamos a ver en este curso. 

De momento, confórmate con que la fórmula para obtener el número de combinaciones con repetición de longitud k para un alfabeto de longitud n es:

$\frac{(k+n-1)!}{k!(n-1)!}$

## Sorpresa! 🤡
Volviendo entonces a la pregunta original, ¿cómo es de sorprendente que `"ACTAT"` aparezca en la secuencia `"ACAACTATGCATACTATCGGGAACTATCCT"` 3 veces?

Para dar con la respuesta, primero tendríamos que calcular la probabilidad de generar el k-mero `"ACTAT"`.
Podemos calcularlo también empíricamente con ayuda de `itertools`:

In [8]:
permutaciones  = list(product('ACGT', repeat=5)) # Todas las permutaciones sin repetición de longitud 5 sobre 4 elementos:

In [9]:
len(permutaciones)

1024

Efectivamente, esto concuerda con lo esperado teóricamente: $n^{k} = 4^{5} = 1024$!!

Ahora piensa, ¿cuántas "oportunidades" tenemos de generar este 5-mero en una cadena de longitud 30?

In [10]:
contador_oportunidades = 0
for i in range(len(seq_1) - 5 + 1):
    contador_oportunidades+=1
print(contador_oportunidades)

26


Por tanto, tiene 26 (n-k+1) oportunidades (cada una con probabilidad 1/1024) de aparecer en esta secuencia!
Esto es, $\frac{1}{26\times4^{5}} = \frac{1}{26624}$. Osea, un poco más de 1 entre 30.000!!

Sí, además tiene que salir otras dos veces, en el primer intento tendríamos 25, o sea, $(n-1)-k+1$ oportunidades (ya que una ya la habríamos "gastado" en que nos salga una bien).Y análogamente, para el segundo caso, tendríamos 24 oportunidades (habríamos gastado ya dos).

En definitiva, la probabilidad combinada de que se den estos tres eventos es:

$\frac{26}{4^{5}}\times\frac{25}{4^{5}}\times\frac{24}{4^{5}} = \frac{26\times 25\times 24}{4^{15}} = 0.00001452863216$. 

Osea, una entre 4 billones!!! Para que te hagas una idea, hay estudios que dicen que la probabilidad de que te caiga un rayo en toda tu vida es, aproximadamente, 1 entre 3.000.000. 

¡Pues que `"ACTAT"` aparezca tres veces en `"ACAACTATGCATACTATCGGGAACTATCCT"` es, más o menos, igual de probable que que te caiga un rayo!
